# CNN for Waste Analysis
With two classes: organic and inorganic

## Prerequisites

`pip install torch torchvision`

- pytorch (for CNN)
- torchvision (for vision processing)

In [1]:
DEBUG = False

## Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  # Import functional module
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## CNN Data Structure

In [3]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)  # Adjusted based on input size
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)  # Adjusted flattening
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Hyperparameters

In [4]:
batch_size = 32
learning_rate = 0.001
num_epochs = 1

## Data Transformation

In [5]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images
    transforms.ToTensor(),            # Convert images to tensor
])

## Load the Data

In [7]:
train_dataset = datasets.ImageFolder(root='../dataset/train', transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)  # Drop last batch if smaller

## Initialize the Model

In [8]:
num_classes = len(train_dataset.classes)
if DEBUG:
    print(f'Number of classes: {num_classes}')

In [9]:
model = CNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Training Loop

In [11]:
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        
        # Print shapes for debugging
        if DEBUG:
            print(f"Output shape: {outputs.shape}, Labels shape: {labels.shape}")
        
        # Ensure the output and labels have the same size
        if outputs.size(0) != labels.size(0):
            if DEBUG:
                print(f"Skipping batch due to size mismatch: outputs {outputs.size(0)}, labels {labels.size(0)}")
            continue
        
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

torch.save(model.state_dict(), 'trained_on_el304_prelim.pth')
print("Model saved as trained_on_el304_prelim.pth")


Epoch [1/1], Loss: 0.2899
Model saved as trained_on_el304_prelim.pth


## Model Evaluation

In [12]:
test_dataset = datasets.ImageFolder(root='../dataset/train', transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation
    for images, labels in test_loader:
        outputs = model(images)  # Forward pass
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
        total += labels.size(0)  # Total number of labels
        correct += (predicted == labels).sum().item()  # Count correct predictions

accuracy = 100 * correct / total
print(f'Correct guesses: {correct}')
print(f'Total guesses: {total}')
print(f'Accuracy of the model on the test dataset: {accuracy:.2f}%')

Correct guesses: 4168%
Total guesses: 4752%
Accuracy of the model on the test dataset: 87.71%
